### Our main objective for this code is to find segments that vary in payment amounts/behaviour the assumption being that each has a stable segment level performance, which is a reasonable assumption because we have cohorts that are the same with respect to the segment/model score.

We are working with synthetic data, so there are distributions and such that doesn't follow the norm for real data, despite this - building the scaffolding for creating a CLV will still be pertinent.

Below is a class that loads the synthetic data, and maps the columns CustomerID,TransactionDate,TransactionAmount to the correct name. There are also methods in this class for creating RFM measures, RFM Segments, Demographics and Transaction Descriptor features (which I dont use), if you can do something useful with it then by all means.

Please Note that the dataloader, and rfm methods are general methods they can be used with any data that contains the columns CustomerID,TransactionID,TransactionDate,TransactionAmount.

I will close off by saying that the point of this exercise is to end off with segments that have stable performance at the "crowd" level. Thus nullifying the problem of high variance when trying to do a regression.

Once we have these segments, we have a choice of 2, we will then extrapolate payments per cohort/crowd/group. That will be the next step, this work will be carried out in a seperate notebook and class with written for the Vintage object with optimisation methods.

In [1]:
import pandas as pd
from pathlib import Path
import sklearn
import category_encoders as ce
import xgboost as xgb
import optuna
import matplotlib.pyplot as plt
import numpy as np
import rfmbinner

class DataLoader:
    
    def __init__(self,path,customer_id,transaction_id,transaction_date,amount):
        """
        Initialize the DataLoader.

        Parameters:
            path (str): Path to the CSV file.
            customer_id (str): Column name for customer IDs.
            transactionid (str): Column name for transaction IDs.
            transaction_date (str): Column name for transaction dates.
            amount (str): Column name for transaction amounts.

        """
        self.path = path
        self.customer_id = customer_id
        self.transaction_id = transaction_id
        self.transaction_date = transaction_date 
        self.amount = amount 
    
    def fetch_data(self) -> pd.DataFrame:
        """
        Fetch raw CSV data from the specified path.

        Parameters:
            path (str): Path to the CSV file.

        Returns:
            pd.DataFrame: Loaded data.
        """
        data_path = Path(self.path)
        if not data_path.exists():
            raise FileNotFoundError(f"CSV file not found at: {data_path}")
        data = pd.read_csv(data_path)
        data.rename(columns={self.customer_id:'CustomerID',
                             self.transaction_id: 'TransactionID',
                             self.transaction_date: 'Date',
                             self.amount: 'Amount'}, inplace=True)
        return data
    
    def calculate_rfm(self, snapshot_date: str, window: pd.Timedelta,df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculate RFM metrics for customer segmentation.

        Parameters:
            snapshot_date (str): Date to calculate recency from (YYYY-MM-DD).

        Returns:
            pd.DataFrame: RFM metrics per CustomerID.
        """
        data = df.copy()
        snapshot = pd.to_datetime(snapshot_date)
        data['Date'] = pd.to_datetime(data.Date)
        data  = data[(data.Date<snapshot) & (data.Date>(snapshot-window))]
        data['recency'] = (snapshot - pd.to_datetime(data['Date'])).dt.days
        print(data)
        rfm = data.groupby('CustomerID').agg({
            'recency': 'min',
            'TransactionID': 'count',               # frequency: number of transactions
            'Amount': 'sum'             # monetary: total spend
        }).reset_index()

        rfm.rename(columns={'TransactionID': 'frequency', 'Amount': 'monetary'}, inplace=True)
        rfm['date'] = snapshot_date

        return rfm[['CustomerID', 'date', 'recency', 'frequency', 'monetary']]
    
    def calculate_target(self,date: str, window_size: pd.Timedelta, \
                         repurchase_threshold: float, df: pd.DataFrame) -> pd.DataFrame:
        """
        Calculating the target for the model
        Parameters:
            date (str): Date to calculate recency from (YYYY-MM-DD).
            window_size (pd.Timedelta): Number of days in which to consider repurchase
            repurchase_threshold: Minimum spend to be considered a repurchase
        Returns:
            pd.DataFrame: Target per CustomerID.
        """
        #caculate whether a customer made total purchases after date exceeding repurchase threshold
        data = df.copy()
        data['Date'] = pd.to_datetime(data['Date'])
        date = pd.to_datetime(date)
        future_purchases = data[(data['Date'] > date) & \
                                (data['Date'] <= date + window_size)]

        target_customers = future_purchases.groupby('CustomerID')[['Amount']].sum()
        target_customers = target_customers[target_customers >= repurchase_threshold].index

        total_future_purchases = future_purchases.groupby('CustomerID')[['Amount']].sum()
        data = data.merge(total_future_purchases.rename(columns={'Amount': 'subsequent_purchases'}),
                            on='CustomerID', how='left')
        data['subsequent_purchases'] = data['subsequent_purchases'].fillna(0)


        data['target'] = data['CustomerID'].isin(target_customers).astype(int)
        data['date'] = date
        return data[['CustomerID','date','target','subsequent_purchases']].drop_duplicates()
    
    def rfm_segments(self,date: str, window: pd.Timedelta, df: pd.DataFrame) -> pd.DataFrame:
        """
        Segments customers based on their RFM score.

        Parameters:
            date (str): Date to calculate recency from (YYYY-MM-DD).
            df (pd.DataFrame): Raw transactional data.

        Returns:
            pd.DataFrame: RFM segments per CustomerID.
        """
        data = df.copy()
        data = self.calculate_rfm(date,window,data)
        # Create RFM scores
        data['r_score'] = pd.qcut(data['recency'].rank(method='first'), 5, labels=[5, 4, 3, 2, 1])
        data['f_score'] = pd.qcut(data['frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
        data['m_score'] = pd.qcut(data['monetary'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])
        data['rfm_score'] = data['r_score'].astype(str) + \
                            data['f_score'].astype(str) + \
                            data['m_score'].astype(str)
        data['rfm_score_int'] = data['r_score'].astype(int) + \
                            data['f_score'].astype(int)+ \
                            data['m_score'].astype(int)           
        #create a dicitionary with the rfm_score that maps into segment, the key must be a regex
        segment_map = {
                    r'^55[1-5]$': 'Champions',
                    r'^54[1-5]$': 'Loyal Customers',
                    r'^45[1-5]$': 'Potential Loyalists',
                    r'^53[1-5]$': 'New or Returning Customers',
                    r'^33[1-5]$': 'Promising',
                    r'^22[1-5]$': 'Needs Attention',
                    r'^\d{3}$': 'Others'  # Matches any other 3-digit score
                }
        data['segment'] = data['rfm_score'].replace(segment_map, regex=True)
        data['date'] = date
        return data[['CustomerID', 'date', 'rfm_score','rfm_score_int', 'segment']]
    
    def dedup_demographic_variables(self,df: pd.DataFrame) -> pd.DataFrame:
        """
        Deduplicate demographic variables for each customer, keeping the earliest entry.

        Parameters:
            df (pd.DataFrame): Raw transactional data with demographic information.

        Returns:
            pd.DataFrame: Deduplicated demographic data per CustomerID.
        """
        data = df.copy()
        data['Date'] = pd.to_datetime(data['Date'])
        
        # Sort by CustomerID and Date to get the most recent demographic info
        data = data.sort_values(by=['CustomerID', 'Date'], ascending=True)
        
        # Drop duplicates, keeping the last (most recent) entry for each CustomerID
        # Assuming demographic variables are 'Gender', 'Age', 'Age Group', extend to the actual list
        demographic_cols = ['CustomerID', 'Gender', 'Age','Province']
        deduplicated_demographics = data[demographic_cols].drop_duplicates(subset=['CustomerID'], keep='first')
        
        return deduplicated_demographics
    
    def transaction_descriptor_variables(self,date) -> pd.DataFrame:

        data = self.fetch_data()
        data['Date'] = pd.to_datetime(data['Date'])
        purchases = data[data.Date < date]
        summary = purchases.groupby('CustomerID')[['ProductCategory','PurchaseChannel',
                                       'PaymentMethod','Store']].agg(pd.Series.mode).reset_index()
        summary['date'] = date
        # Rename columns for clarity
        summary.rename(columns={'ProductCategory': 'Most_frequented_Category',
                                  'PurchaseChannel': 'Most_frequented_Channel',
                                  'PaymentMethod': 'Most_used_payment_method',
                                  'Store': 'Most_frequented_Store'}, inplace=True)
        return summary[['CustomerID','date','Most_frequented_Channel','Most_frequented_Category','Most_used_payment_method','Most_frequented_Store']]
    
    def dedup_demographic_variables(self,df: pd.DataFrame) -> pd.DataFrame:
        """
        Deduplicate demographic variables for each customer, keeping the latest entry.

        Parameters:
            df (pd.DataFrame): Raw transactional data with demographic information.

        Returns:
            pd.DataFrame: Deduplicated demographic data per CustomerID.
        """
        data = df.copy()
        data['Date'] = pd.to_datetime(data['Date'])
        
        # Sort by CustomerID and Date to get the most recent demographic info
        data = data.sort_values(by=['CustomerID', 'Date'], ascending=True)
        
        # Drop duplicates, keeping the last (most recent) entry for each CustomerID
        # Assuming demographic variables are 'Gender', 'Age', 'Age Group', extend to the actual list
        demographic_cols = ['CustomerID', 'Gender', 'Age','Province']
        deduplicated_demographics = data[demographic_cols].drop_duplicates(subset=['CustomerID'], keep='last')
        
        return deduplicated_demographics
    


d:\VS CODE PROJECTS\CLVM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


For the code below the process is as follows:

1. Initialise a dataloader
2. Fetch the synthetic data
3. Run the appropriate methods of the dataloader in order to create a collection of features
4. We join all the features together and create a "model_data" dataframe.

In [2]:
params = {'path':r'data\synthetic_transactions.csv',
            'customer_id':'CustomerID',
            'transaction_id':'TransactionID',
            'transaction_date':'TransactionDate',
            'amount':'TransactionAmount'}
loader = DataLoader(**params)
data = loader.fetch_data()

In [3]:
start_date = '2022-08-30'
end_date = pd.to_datetime('2025-08-07')
pivot_date = pd.to_datetime(start_date)
rfms = []
targets = []
rfm_segments = []
transaction_descriptor_data = []

while pivot_date<end_date:
    rfms.append(loader.calculate_rfm(pivot_date,pd.Timedelta(weeks=4),data))
    rfm_segments.append(loader.rfm_segments(pivot_date,pd.Timedelta(weeks=4),data))
    targets.append(loader.calculate_target(pivot_date,pd.Timedelta(weeks=12),0,data))
    pivot_date+=pd.Timedelta(weeks=4)

rfm_data = pd.concat(rfms)
target_data = pd.concat(targets)
rfm_segments = pd.concat(rfm_segments)
dem_data = loader.dedup_demographic_variables(data)

       CustomerID  TransactionID                Date  Amount  ProductCategory  \
86     CUST-01373  TRANS-0000087 2022-08-20 09:18:26  653.94        Groceries   
107    CUST-02392  TRANS-0000108 2022-08-14 08:55:30  704.37      Electronics   
145    CUST-09188  TRANS-0000146 2022-08-19 19:03:48   18.79         Clothing   
171    CUST-04213  TRANS-0000172 2022-08-19 14:44:35    8.36        Groceries   
237    CUST-09536  TRANS-0000238 2022-08-21 20:34:15   12.31            Books   
...           ...            ...                 ...     ...              ...   
99784  CUST-03672  TRANS-0099785 2022-08-24 16:25:37  169.28           Sports   
99798  CUST-03887  TRANS-0099799 2022-08-17 00:33:14  568.03  Health & Beauty   
99813  CUST-00152  TRANS-0099814 2022-08-10 14:31:02  234.13      Electronics   
99932  CUST-08362  TRANS-0099933 2022-08-11 21:36:35  515.39  Health & Beauty   
99970  CUST-04278  TRANS-0099971 2022-08-09 23:34:28   47.13           Sports   

       Age       Province  

In [4]:
model_data = rfm_data.merge(target_data,how='left',left_on=['CustomerID','date'],right_on=['CustomerID','date'])
model_data = model_data.merge(dem_data,how='left',left_on='CustomerID',right_on='CustomerID')
#model_data = model_data.merge(transaction_descriptor_data,how='left', left_on=['CustomerID','date'],right_on=['CustomerID','date'])
model_data = model_data.merge(rfm_segments,how='left', left_on=['CustomerID','date'],right_on=['CustomerID','date'])

In [5]:
end_date_of_training = '2024-07-31'
model_data = model_data.set_index(['CustomerID','date']).drop('rfm_score',axis=1)
model_data = model_data[model_data.index.get_level_values(1)<end_date_of_training]

In [6]:
model_data.target.value_counts()/len(model_data)

target
1    0.53721
0    0.46279
Name: count, dtype: float64

In [7]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    model_data.drop(['target', 'subsequent_purchases'], axis=1),
    model_data['target'],
    test_size=0.2,
    random_state=42
)

for col in X_train.columns:
    sample = X_train[col].iloc[0]
    if isinstance(sample, (list, np.ndarray)):
        print(f"Column '{col}' contains unhashable types: {type(sample)}")

encoder = ce.TargetEncoder()
X_train = encoder.fit_transform(X_train, y_train)
X_test = encoder.transform(X_test)

# Optuna + XGBoost training
def train_xgboost_with_optuna(X, y, n_trials=50, test_size=0.2, random_state=42):
    X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    def objective(trial):
        params = {
            "verbosity": 0,
            "objective": "binary:logistic",
            "eval_metric": "logloss",
            "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
            "lambda": trial.suggest_float("lambda", 1e-8, 10.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-8, 10.0, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "eta": trial.suggest_float("eta", 0.01, 0.3),
        }

        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        model = xgb.train(params, dtrain, num_boost_round=100,
                          evals=[(dval, "validation")],
                          early_stopping_rounds=10,
                          verbose_eval=False)
        preds = model.predict(dval)
        return sklearn.metrics.roc_auc_score(y_val, preds)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params
    best_params.update({
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "logloss"
    })

    final_model = xgb.train(best_params, xgb.DMatrix(X, label=y), num_boost_round=study.best_trial.number)
    return final_model, study

# Train and evaluate
model, study = train_xgboost_with_optuna(X_train, y_train)
y_probs = model.predict(xgb.DMatrix(X_test))

print(sklearn.metrics.classification_report(y_test, y_probs > 0.5))
print("ROC AUC:", sklearn.metrics.roc_auc_score(y_test, y_probs))


[I 2025-08-09 12:55:58,657] A new study created in memory with name: no-name-93ddbbf2-7790-44d2-a754-ec305a727c1a
[I 2025-08-09 12:55:58,913] Trial 0 finished with value: 0.5047407128619951 and parameters: {'booster': 'gbtree', 'lambda': 0.0001618690618020874, 'alpha': 0.00468600827686352, 'subsample': 0.7313430954968676, 'colsample_bytree': 0.7690398792705446, 'max_depth': 3, 'eta': 0.12553397181798762}. Best is trial 0 with value: 0.5047407128619951.
[I 2025-08-09 12:55:58,942] Trial 1 finished with value: 0.4968222855785222 and parameters: {'booster': 'gbtree', 'lambda': 2.1175565760091343e-06, 'alpha': 0.045064713889643046, 'subsample': 0.9854353626625947, 'colsample_bytree': 0.5266768281209053, 'max_depth': 5, 'eta': 0.018053725759298228}. Best is trial 0 with value: 0.5047407128619951.
[I 2025-08-09 12:55:59,005] Trial 2 finished with value: 0.5115319475622997 and parameters: {'booster': 'gbtree', 'lambda': 4.591930736016293e-06, 'alpha': 0.006946200039189061, 'subsample': 0.7743

              precision    recall  f1-score   support

           0       0.47      0.07      0.12      5312
           1       0.54      0.94      0.69      6330

    accuracy                           0.54     11642
   macro avg       0.51      0.50      0.40     11642
weighted avg       0.51      0.54      0.43     11642

ROC AUC: 0.5048517381135917


In [8]:
sklearn.metrics.roc_auc_score(y_test,model.predict(xgb.DMatrix(X_test)))

0.5048517381135917

In [9]:
X_test['QSegment'] = pd.qcut(y_probs,10)

In [10]:
summary_data = X_test.merge(target_data,how='left', left_on='CustomerID', right_on='CustomerID')
summary = summary_data.groupby(['QSegment']).agg({'recency':'count','monetary':'sum','target':'sum','subsequent_purchases':'sum'})
summary = summary.rename(columns = {'recency':'Count','monetary':'prior_purchases','target':'Purchase Rate'})
summary['prior_purchases in ZAR'] = summary.prior_purchases.apply(lambda x: f'R {x:,.2f}')
summary['Value %'] = (summary['prior_purchases']/summary['prior_purchases'].sum()*100).apply(lambda x: f'{x:.2f}%')
summary['AVG subs. spend in ZAR'] = (summary['subsequent_purchases']/summary.Count).apply(lambda x: f'R {x:.2f}')
summary['subsequent_purchases in ZAR (3 month window)'] = summary.subsequent_purchases.apply(lambda x: f"R {x:,.2f}")
summary['Purchase Rate'] =  (summary['Purchase Rate']/summary.Count).apply( lambda x: f'{x:.2%}')
summary.drop(['prior_purchases','subsequent_purchases'],axis=1)

C:\Users\wesle\AppData\Local\Temp\ipykernel_47944\2678178072.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  summary = summary_data.groupby(['QSegment']).agg({'recency':'count','monetary':'sum','target':'sum','subsequent_purchases':'sum'})


,Count,Purchase Rate,prior_purchases in ZAR,Value %,AVG subs. spend in ZAR,subsequent_purchases in ZAR (3 month window)
QSegment,,,,,,
"(0.362, 0.507]",45435,54.96%,"R 32,167,262.40",12.60%,R 410.98,"R 18,672,935.77"
"(0.507, 0.518]",45435,54.85%,"R 26,305,397.43",10.30%,R 400.57,"R 18,199,827.14"
"(0.518, 0.525]",45357,55.19%,"R 25,818,380.25",10.11%,R 400.13,"R 18,148,514.36"
"(0.525, 0.531]",45396,55.08%,"R 24,301,748.64",9.52%,R 389.79,"R 17,694,797.68"
"(0.531, 0.537]",45396,55.05%,"R 25,289,759.43",9.90%,R 400.43,"R 18,177,716.95"
"(0.537, 0.542]",45396,55.54%,"R 24,947,143.65",9.77%,R 407.66,"R 18,506,171.74"
"(0.542, 0.548]",45396,54.67%,"R 25,644,755.76",10.04%,R 398.89,"R 18,107,889.97"
"(0.548, 0.555]",45396,56.14%,"R 23,411,127.87",9.17%,R 411.83,"R 18,695,411.84"
"(0.555, 0.566]",45396,55.05%,"R 24,613,683.12",9.64%,R 400.95,"R 18,201,669.26"


In [11]:
summary_data = X_test.merge(target_data,how='left', left_on='CustomerID', right_on='CustomerID')
summary = summary_data.groupby(['rfm_score_int']).agg({'recency':'count','monetary':'sum','target':'sum','subsequent_purchases':'sum'})
summary = summary.rename(columns = {'recency':'Count','monetary':'prior_purchases','target':'Purchase Rate'})
summary['prior_purchases in ZAR'] = summary.prior_purchases.apply(lambda x: f'R {x:,.2f}')
summary['Value %'] = (summary['prior_purchases']/summary['prior_purchases'].sum()*100).apply(lambda x: f'{x:.2f}%')
summary['AVG subs. spend in ZAR'] = (summary['subsequent_purchases']/summary.Count).apply(lambda x: f'R {x:.2f}')
summary['subsequent_purchases in ZAR (3 month window)'] = summary.subsequent_purchases.apply(lambda x: f"R {x:,.2f}")
summary['Purchase Rate'] =  (summary['Purchase Rate']/summary.Count).apply( lambda x: f'{x:.2%}')
summary.drop(['prior_purchases','subsequent_purchases'],axis=1)

,Count,Purchase Rate,prior_purchases in ZAR,Value %,AVG subs. spend in ZAR,subsequent_purchases in ZAR (3 month window)
rfm_score_int,,,,,,
3,4212,56.74%,"R 250,986.06",0.10%,R 386.23,"R 1,626,819.87"
4,13182,55.42%,"R 1,399,187.01",0.55%,R 371.55,"R 4,897,781.64"
5,26520,55.19%,"R 4,286,784.45",1.68%,R 376.06,"R 9,973,026.67"
6,39351,54.52%,"R 8,835,063.12",3.46%,R 366.52,"R 14,422,784.06"
7,58071,55.14%,"R 19,365,327.54",7.58%,R 386.40,"R 22,438,393.20"
8,65442,54.39%,"R 27,690,536.64",10.84%,R 382.19,"R 25,011,602.43"
9,63258,55.55%,"R 32,968,830.57",12.91%,R 401.71,"R 25,411,094.19"
10,56043,55.50%,"R 35,190,040.47",13.78%,R 411.41,"R 23,056,501.91"
11,46176,54.93%,"R 36,637,036.80",14.35%,R 414.59,"R 19,144,254.29"


In [12]:
pd.DataFrame(data = zip(X_train.columns,model.get_score(importance_type='gain').values()),columns=['Features','Importances'])

,Features,Importances
0,recency,4.522341
1,frequency,4.662859
2,monetary,5.038667
3,Gender,4.211041
4,Age,4.744610
5,Province,4.851005
6,rfm_score_int,4.511037
7,segment,4.316330


In [13]:
model_data.segment.unique()

array(['Others', 'Champions', 'Potential Loyalists', 'Needs Attention',
       'New or Returning Customers', 'Promising', 'Loyal Customers'],
      dtype=object)